In [7]:
import numpy as np

ntsteps = 3

# Load data
predict = np.load("predictions.npy")  # shape = (batch_no, loc/vel, dataset_in_batch, n_nodes, timesteps, 3_directions)
print(predict.shape)
nDataset = predict.shape[0]//ntsteps

data = np.zeros(shape=(nDataset,ntsteps,predict.shape[1],predict.shape[2]))
print(nDataset)

l = 0 
for i in range(ntsteps):
  data[i,:,:,:] = predict[l:l+3,:,:]
  l += ntsteps

data.shape

coords = data[:, :, :, 0:3]  # Extract coordinates
vel = data[:, :, :, 3:6]  # Extract velocities
connect = np.load("connectivity.npy")  # Load connectivity data

# Define the Tecplot file writing function
def write_tecplot_dat(filename, coords, vel, connectivity):
  n_nodes = coords.shape[1]
  n_elements = connectivity.shape[0]

  for t in range(ntsteps):
    with open(filename+'%i.dat'%t, 'w') as f:
      f.write('TITLE="3D TRIANGULAR SURFACE DATA"\n')
      f.write('VARIABLES="X","Y","Z"\n')
      f.write(f'ZONE T="unstruc" N={n_nodes} E={n_elements} F=FEPOINT ET=TRIANGLE\n')

      # Write node data
      for i in range(n_nodes):
        x, y, z = coords[t, i, :]
        u, v, w = vel[t, i, :]
        f.write(f'{x} {y} {z}\n')

      # Write connectivity data
      for conn in connectivity:
        f.write(f'{conn[0] + 1} {conn[1] + 1} {conn[2] + 1}\n')  # Tecplot uses 1-based indexing

# Write the data to a Tecplot .dat file
for t in range(nDataset):   
  write_tecplot_dat("./Post_Proc_Data/membrane/%i_"%t, coords[t,:,:,:], vel[t,:,:,:], connect)

(57, 1757, 6)
19


In [56]:
"""
This is the script to visualize vorticity contour
"""
import numpy as np
import matplotlib.pyplot as plt

# data = np.load("./sample_data/data.npy")
tstep = 1
sliceID = 0

# Extract the specific slice of data
x = data[tstep, 0, :, :, :]
y = data[tstep, 1, :, :, :]
z = data[tstep, 2, :, :, :]
# vorticity = data[tstep, 3, :, :, sliceID]
# # Plot the contour
# plt.figure(1)
# contour = plt.contourf(x, y, vorticity, cmap='viridis')  # Use a colormap like 'viridis'
# plt.colorbar(contour)  # Add a color bar for reference
# plt.xlabel('X-axis')
# plt.ylabel('Y-axis')
# plt.title('Vorticity Contour')
# plt.show()

In [57]:
z[0,0,50]

20.60465431213379

In [61]:
import os
import numpy as np
import vtk
from vtk.util import numpy_support

# Directory containing the VTI files

def load_vti_file(file_name):
		reader = vtk.vtkXMLImageDataReader()
		reader.SetFileName(file_name)
		reader.Update()
		return reader.GetOutput()

def extract_data(image_data):
		X_array = image_data.GetPointData().GetArray("X")
		Y_array = image_data.GetPointData().GetArray("Y")
		Z_array = image_data.GetPointData().GetArray("Z")
		vorticity_array = image_data.GetPointData().GetArray("Vorticity")

		num_points = image_data.GetNumberOfPoints()
		print(num_points)
		X = np.zeros(num_points)
		Y = np.zeros(num_points)
		Z = np.zeros(num_points)
		vorticity = np.zeros(num_points)
		temperature = np.zeros(num_points)

		for i in range(num_points):
				# velocity = velocity_array.GetTuple(i)
				# u_array[i] = velocity[0]
				# v_array[i] = velocity[1]
				# w_array[i] = velocity[2]
				X[i] = X_array.GetValue(i)
				Y[i] = Y_array.GetValue(i)
				Z[i] = Z_array.GetValue(i)
				vorticity[i] = vorticity_array.GetValue(i)

		return X, Y, Z, vorticity

def reshape_arrays_to_grid(X, Y, Z, vorticity, temperature, x_res, y_res, z_res):
    x_reshaped = np.reshape(X, (z_res, y_res, x_res))
    y_reshaped = np.reshape(Y, (z_res, y_res, x_res))
    z_reshaped = np.reshape(Z, (z_res, y_res, x_res))
    vorticity_reshaped = np.reshape(vorticity, (z_res, y_res, x_res))
    return x_reshaped, y_reshaped, z_reshaped, vorticity_reshaped


vti_directory = './sample_data/vtiData/'  # Change this to your VTI files directory
ntsteps = 201
nFeatures = 4
nx = 70
ny = 68
nz = 59

# file_name = vti_directory + "data_0.vti"
# image_data = load_vti_file(file_name)

# x, y, z, vorticity = extract_data(image_data)

# x = np.reshape(x, (nx, ny, nz), order='F')
# y = np.reshape(y, (nx, ny, nz), order='F')
# z = np.reshape(z, (nx, ny, nz), order='F')
# vorticity = np.reshape(vorticity, (nx, ny, nz), order='F')

# levels = np.linspace(0, 10, 11)

# plt.figure(figsize=(8, 6))
# plt.contourf(x[:, :, 50], y[:, :, 50], vorticity[:, :, 50], cmap='YlOrRd',levels=levels)
# plt.colorbar(label='Temperature (C)')
# plt.xlabel('X')
# plt.ylabel('Z')
# plt.title('Temperature Contour at y = 12')
# plt.grid(True)
# plt.axis('equal')
# plt.savefig("temp.png")


data_list = []
for tstep in range(ntsteps):
	file = vti_directory+"data_%i.vti"%tstep
	print(file)
	vtiData = load_vti_file(file)
	listData = extract_data(vtiData)
	arrayData = np.zeros(shape=(nFeatures,nx,ny,nz))

	for i in range(nFeatures):
		arrayData[i] = np.reshape(listData[i], (nx, ny, nz), order='F')
	
	data_list.append(arrayData)

data = np.array(data_list)
# np.save('./sample_data/data.npy', np_data)


#     # Create a reader
#     reader = vtk.vtkXMLImageDataReader()
#     reader.SetFileName(os.path.join(vti_directory, file))
#     reader.Update()

#     # Get the output data object
#     imageData = reader.GetOutput()

#     # If data exists, extract it
#     if imageData:
#         # Extract the data arrays
#         num_arrays = imageData.GetPointData().GetNumberOfArrays()
#         temp_data = []
#         for i in range(num_arrays):
#             array = imageData.GetPointData().GetArray(i)
#             if array:
#                 np_array = numpy_support.vtk_to_numpy(array)
#                 # Reshape the array to match the grid dimensions (nx, ny, nz)
#                 dims = imageData.GetDimensions()
#                 np_array = np_array.reshape(dims)
#                 temp_data.append(np_array)

#         data_list.append(temp_data)

# # Convert list to numpy array, ensuring non-empty
# if data_list:
#     np_data = np.array(data_list)
# else:
#     np_data = np.array([])

# # Check shape - should be [ntsteps, nData, nx, ny, nz]
# print(np_data.shape)

# # Save the numpy array to a file
# np.save('./sample_data/data.npy', np_data)


./sample_data/vtiData/data_0.vti
280840
./sample_data/vtiData/data_1.vti
280840
./sample_data/vtiData/data_2.vti
280840
./sample_data/vtiData/data_3.vti
280840
./sample_data/vtiData/data_4.vti
280840
./sample_data/vtiData/data_5.vti
280840
./sample_data/vtiData/data_6.vti
280840
./sample_data/vtiData/data_7.vti
280840
./sample_data/vtiData/data_8.vti
280840
./sample_data/vtiData/data_9.vti
280840
./sample_data/vtiData/data_10.vti
280840
./sample_data/vtiData/data_11.vti
280840
./sample_data/vtiData/data_12.vti
280840
./sample_data/vtiData/data_13.vti
280840
./sample_data/vtiData/data_14.vti
280840
./sample_data/vtiData/data_15.vti
280840
./sample_data/vtiData/data_16.vti
280840
./sample_data/vtiData/data_17.vti
280840
./sample_data/vtiData/data_18.vti
280840
./sample_data/vtiData/data_19.vti
280840
./sample_data/vtiData/data_20.vti
280840
./sample_data/vtiData/data_21.vti
280840
./sample_data/vtiData/data_22.vti
280840
./sample_data/vtiData/data_23.vti
280840
./sample_data/vtiData/data